In [1]:
!wget https://raw.githubusercontent.com/Mehrdadghassabi/ngram_next_token_prediction/main/divar_posts_dataset.csv
!pip install hazm

--2024-08-04 20:09:50--  https://raw.githubusercontent.com/Mehrdadghassabi/ngram_next_token_prediction/main/divar_posts_dataset.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30743 (30K) [text/plain]
Saving to: ‘divar_posts_dataset.csv.1’

divar_posts_dataset 100%[===================>]  30.02K  --.-KB/s    in 0.04s   

2024-08-04 20:09:50 (841 KB/s) - ‘divar_posts_dataset.csv.1’ saved [30743/30743]



In [2]:
import hazm
import csv
import re
from collections import defaultdict
import math
import random

In [3]:
with open('divar_posts_dataset.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    comments = []
    for row in reader:
        comments.append(row['comment'])
print(comments[1])

سلام,یک عدد گلدون نخل سه طبقه ی سالم دارم با پایه ی سفالی به علت جابجایی میفروشمش


In [4]:
def get_sentences(text):
    tokenizer = hazm.SentenceTokenizer()
    sentences = tokenizer.tokenize(text)
    return sentences

comments_sentences = []
for comment in comments:
    sentences = get_sentences(comment)
    comments_sentences.append(sentences)
print(comments_sentences[17])

['تلوزیون رنگی ۲۹ اینچ توشیبا صفحه تخت کاملا سالم همراه با میز فابریک']


In [5]:
def remove_punctuation(text):
    persian_punctuation = 'ء\n()/•٪,،؛؟«»٬:.!-…*_×''+$'
    cleaned_text = ""
    for char in text:
        if char in persian_punctuation:
           cleaned_text = cleaned_text + '\u200C'
        else:
           cleaned_text = cleaned_text + char

    return cleaned_text

In [6]:
def replace_persian_numbers(text):
    persian_numbers = {
        '۰': '0',
        '۱': '1',
        '۲': '2',
        '۳': '3',
        '۴': '4',
        '۵': '5',
        '۶': '6',
        '۷': '7',
        '۸': '8',
        '۹': '9'
    }

    for persian_num, latin_num in persian_numbers.items():
        text = text.replace(persian_num, latin_num)
    text = re.sub(r'\d+', '$Num', text)

    return text

In [7]:
def transliterate(text):
    persian_alphabet = {
        'a': 'ا',
        'b': 'ب',
        'c': 'س',
        'd': 'د',
        'e': '',
        'f': 'ف',
        'g': 'گ',
        'h': 'ه',
        'i': 'ی',
        'j': 'ج',
        'k': 'ک',
        'l': 'ل',
        'm': 'م',
        'n': 'ن',
        'o': '',
        'p': 'پ',
        'q': 'ک',
        'r': 'ر',
        's': 'س',
        't': 'ت',
        'u': 'و',
        'v': 'و',
        'w': 'و',
        'x': 'کس',
        'y': 'ی',
        'z': 'ز'
    }

    result = ''
    for char in text:
        if char.lower() in persian_alphabet:
            result += persian_alphabet[char.lower()]
        else:
            result += char

    return result

In [8]:
def normalize_comments(comments):
    normalizer = hazm.Normalizer()
    normalized_comments = []
    for comment in comments:
        comment = normalizer.normalize(comment)
        comment = remove_punctuation(comment)
        comment = comment.replace('\u200C', " ")
        comment = transliterate(comment)
        comment = replace_persian_numbers(comment)
        comment = comment.replace("$Num", "")
        comment = comment.replace("  ", " ")
        normalized_comments.append(comment)
    return normalized_comments

In [9]:
normalized_comments = normalize_comments(comments)
print(normalized_comments[0])

کلاسیک و شیک و استثنایی چرم مالزی چوب راش فوق العاده سالم و بدون عیب و ایراد


In [10]:
def build_ngram_model(comments, n):
    ngrams = defaultdict(int)
    total_ngrams = 0

    for comment in comments:
        tokenizer = hazm.WordTokenizer()
        words = tokenizer.tokenize(comment)

        for i in range(len(words) - n + 1):
            ngram = tuple(words[i:i+n])
            ngrams[ngram] += 1
            total_ngrams += 1

    ngram_model = {}
    for ngram, count in ngrams.items():
        ngram_model[ngram] = count

    return ngram_model,total_ngrams

In [11]:
def get_ngram_count(ngram_model,ngram):
    try:
      return ngram_model[ngram]
    except:
      return 0

In [12]:
unigram_model,total_unigrams = build_ngram_model(comments = normalized_comments, n = 1)
unigram_model = dict(sorted(unigram_model.items(), key = lambda x:-x[1]))
bigram_model,total_bigrams = build_ngram_model(comments = normalized_comments, n = 2)
bigram_model = dict(sorted(bigram_model.items(), key = lambda x:-x[1]))
trigram_model,total_trigrams = build_ngram_model(comments = normalized_comments, n = 3)
trigram_model = dict(sorted(trigram_model.items(), key = lambda x:-x[1]))

In [13]:
def count_dic(ngram_model):
    dic = {}
    for key in ngram_model.keys():
        if ngram_model[key] not in dic.keys():
           dic[ngram_model[key]] = 1
        else:
           dic[ngram_model[key]] += 1
    return dic

In [14]:
def N(dic,c):
    try:
      return dic[c]
    except:
      return 0.001

In [15]:
def ngram_probablity1(ngram,word,
                     unigram_model,total_unigrams,bigram_model,trigram_model,
                     n):
    assert n>=1 and n<=3
    prob = 0
    V = len(unigram_model.keys())
    K = 0.005
    if n == 1:
       prob = (get_ngram_count(unigram_model,(word,)) + K )/(total_unigrams + K*V)
    if n == 2:
       prob = (get_ngram_count(bigram_model,ngram+(word,)) + K )/(get_ngram_count(unigram_model,ngram) + K*V)
    if n == 3:
       prob = (get_ngram_count(trigram_model,ngram+(word,)) + K )/(get_ngram_count(bigram_model,ngram) + K*V)

    return prob


In [16]:
def choose_uniform(dic):
    if dic == {} or dic == None:
       return choose_uniform(unigram_model)
    total = sum(dic.values())
    cumulative_sum = 0
    rand_num = random.randint(1,total)
    for ngram,count in dic.items():
        cumulative_sum += count
        if rand_num <= cumulative_sum:
           return ngram[0]

In [17]:
def predict_next_token(n,previous_words,unigram_model,bigram_model,trigram_model):
      if n == 1 :
         return choose_uniform(None)
      if n == 2 :
         di = {}
         for ngram,count in bigram_model.items():
             if ngram[0] == previous_words[0]:
                di[(ngram[1],)] = count
         return choose_uniform(di)
      if n == 3 :
         di = {}
         for ngram,count in trigram_model.items():
             if ngram[0] == previous_words[0] and ngram[1] == previous_words[1]:
                di[(ngram[2],)] = count
         return choose_uniform(di)

In [18]:
previous_words = None
predict_next_token(1,previous_words,unigram_model,bigram_model,trigram_model)

'پارچه'

In [19]:
previous_words = ('پیامک',)
predict_next_token(2,previous_words,unigram_model,bigram_model,trigram_model)

'دهید'

In [20]:
previous_words = ('پیامک','جواب')
predict_next_token(3,previous_words,unigram_model,bigram_model,trigram_model)

'داده'

In [21]:
def complete_sentence(sentence,n,max_token = 10):
    tokenizer = hazm.WordTokenizer()
    words = tokenizer.tokenize(sentence)
    num_words = len(words)
    max_token -= num_words
    predicted = sentence
    pw = words[len(words)-1]
    ppw = words[len(words)-2]
    if n == 1:
       for i in range(max_token):
           w = predict_next_token(n,None,unigram_model,bigram_model,trigram_model)
           predicted += ' ' + w
    if n == 2:
       for i in range(max_token):
           previous_words = (pw,)
           w = predict_next_token(n,previous_words,unigram_model,bigram_model,trigram_model)
           predicted += ' ' + w
           pw = w
    if n == 3:
       for i in range(max_token):
           previous_words = (pw,ppw)
           w = predict_next_token(n,previous_words,unigram_model,bigram_model,trigram_model)
           predicted += ' ' + w
           ppw = pw
           pw = w
    return predicted

In [22]:
sentence = 'مبل هفت نفره خود رنگ'
print('unigram prediction: '+str(complete_sentence(sentence,1,max_token = 10)))
print('bigram prediction: '+str(complete_sentence(sentence,2,max_token = 10)))
print('trigram prediction: '+str(complete_sentence(sentence,3,max_token = 10)))

unigram prediction: مبل هفت نفره خود رنگ تلویزیون نو همراه بدون تمام
bigram prediction: مبل هفت نفره خود رنگ سورمه ای مخصوص فلافل حرفه
trigram prediction: مبل هفت نفره خود رنگ نقر محصولات بیمه که مارک


In [23]:
sentence = 'دستگاه تردمیل نو'
print('unigram prediction: '+str(complete_sentence(sentence,1,max_token = 10)))
print('bigram prediction: '+str(complete_sentence(sentence,2,max_token = 10)))
print('trigram prediction: '+str(complete_sentence(sentence,3,max_token = 10)))

unigram prediction: دستگاه تردمیل نو گارانتی تحویل جزئی کارکرد رنگ روزه یا
bigram prediction: دستگاه تردمیل نو سالم و سایز نومویک عدد لاستیک پهن
trigram prediction: دستگاه تردمیل نو تمیزه داخل عدد گانه دیگه دی هم


In [24]:
sentence = 'کفش مردانه'
#print('unigram prediction: '+str(complete_sentence(sentence,1,max_token = 10)))
print('bigram prediction: '+str(complete_sentence(sentence,2,max_token = 100)))
#print('trigram prediction: '+str(complete_sentence(sentence,3,max_token = 10)))

bigram prediction: کفش مردانه اسپرت بیمه سال تخفیف جزئی داده می شود فقط با جعبه سمس جواب داده نمی کنم سر هم میدم در مصرف انرژی طراحی زیبای مسطح و بدون حتی خط خش جمعا ت و تمیز بدون ظربه خرید و یک عدد تخت بسیار تمیز در ماه گارانتی استفاده شده فنر روغنی تعمیری صندلی اسپرت بند مشکی تمیز و دوسیم کارته صدای قوی رم قدرتمند گیگ حافظه داخلی گیگ دوربین و به دکمه های چرب تو عکس مشخصه چقدر تمیزه گلس درجه یک جفت باند پایونیر و بسیار سالم دارم با قابلیت کنترل کارتن نو با گلدان پایه معامله میدم


In [25]:
sentence = 'تعدادی وسایل اداری و پزشکی'
#print('unigram prediction: '+str(complete_sentence(sentence,1,max_token = 10)))
print('bigram prediction: '+str(complete_sentence(sentence,2,max_token = 100)))
#print('trigram prediction: '+str(complete_sentence(sentence,3,max_token = 10)))

bigram prediction: تعدادی وسایل اداری و پزشکی استفاده شده میشه روی جیر تمیز بدون برفک سامانه روشنائی لد طبقه تاشو نگهدارنده بطری تاشو سیستم سرمایش سریع در حد معقول یاحق تلفن هماهنگی نوم نوم تلگرام ممنون از دیوار ایکس باکس وان اکبند اکبند پلمب باز نشده خریدار واقعی تومان کرومیت تومان متری پشم سالم می فروشم تشک نفره جدید با تلگرام و خانه لطفا فقط تماس بگیرید پیامک جواب داده می شود فقط گلگیرها بدون موج رادیو ضبط آیوا نو تاحالا تنش نکرده خودم تا لامپ که بالای قابلمه سفید رینگ اسپرت بیمه تا دارم با تشکر از چوب گردو با زیره
